## Data Loaders

This notebook demonstrates the use of vertex loader in `pyTigerGraph`. The job of a data loader is to pull data from the TigerGraph database. Currently, the following data loaders are provided:
* EdgeLoader, which returns batches of edges.
* VertexLoader, which returns batches of vertices.
* GraphLoader, which returns randomly sampled (probably disconnected) subgraphs in pandas `dataframe`, `PyG` or `DGL` format.
* NeighborLoader, which returns subgraphs using neighbor sampling in `dataframe`, `PyG` or `DGL` format.

Every data loader above can either get all the batches as a HTTP response (default) or stream every batch through Kafka. The former mechanism is good for testing with small graphs and it is fast, but it subjects to a data size limit of 2GB. For large graphs, the HTTP channel will likely fail due to size limit and network connectivity issues. Streaming via Kafka is offered for data robustness and scalability. Also, Kafka excels at multi-consumer use cases, and it is efficient for model search or hyperparameter tuning when there are multiiple consumers of the same data. 

Note: For the data loaders to work, a few UDFs (User Defined Functions) have to be installed into the TigerGraph database.

### Connection to Database

The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. It is able to perform quite a few database tasks. Please see its documentation for details.

In [1]:
from pyTigerGraph import TigerGraphConnection

In [2]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    useCert=False
)

In [3]:
# Number of vertices for every vertex type
conn.getVertexCount('*')

{'Paper': 2708}

In [4]:
# Number of edges for every type
conn.getEdgeCount()

{'Cite': 10556}

### Vertex Loader

VertexLoader pulls batches of vertices from database. Specifically, it divides vertices into `num_batches` and returns each batch separately. The boolean attribute provided to `filter_by` indicates which vertices are included. If you need random batches, set `shuffle` to True.

**Note**: For the first time you initialize the loader on a graph in TigerGraph, the initialization might take a minute as it installs the corresponding query to the database and optimizes it. However, the query installation only needs to be done once, so it will take no time when you initialize the loader on the same TG graph again.

There are two ways to use the data loader. See [here](https://github.com/TigerGraph-DevLabs/mlworkbench-docs/blob/main/tutorials/basics/2_dataloaders.ipynb) for examples.
* First, it can be used as an iterable, which means you can loop through it to get every batch of data. If you load all vertices at once (`num_batches=1`), there will be only one batch (of all the vertices) in the iterator.
* Second, you can access the `data` property of the class directly. If there is only one batch of data to load, it will give you the batch directly instead of an iterator, which might make more sense in that case. If there are multiple batches of data to load, it will return the loader again.

Args:
* attributes (list, optional):
                Vertex attributes to be included. Defaults to None.
* batch_size (int, optional):
                Number of vertices in each batch.
                Defaults to None.
* num_batches (int, optional):
                Number of batches to split the vertices.
                Defaults to 1.
* shuffle (bool, optional):
                Whether to shuffle the vertices before loading data.
                Defaults to False.
* filter_by (str, optional):
                A boolean attribute used to indicate which vertices
                can be included. Defaults to None.
* output_format (str, optional):
                Format of the output data of the loader. Only
                "dataframe" is supported. Defaults to "dataframe".
* loader_id (str, optional):
                An identifier of the loader which can be any string. It is
                also used as the Kafka topic name. If `None`, a random string will be generated
                for it. Defaults to None.
* buffer_size (int, optional):
                Number of data batches to prefetch and store in memory. Defaults to 4.
* kafka_address (str, optional):
                Address of the kafka broker. Defaults to None.
* kafka_max_msg_size (int, optional):
                Maximum size of a Kafka message in bytes.
                Defaults to 104857600.
* kafka_num_partitions (int, optional):
                Number of partitions for the topic created by this loader.
                Defaults to 1.
* kafka_replica_factor (int, optional):
                Number of replications for the topic created by this loader.
                Defaults to 1.
* kafka_retention_ms (int, optional):
                Retention time for messages in the topic created by this
                loader in milliseconds. Defaults to 60000.
* kafka_auto_del_topic (bool, optional):
                Whether to delete the Kafka topic once the
                loader finishes pulling data. Defaults to True.
* kafka_address_consumer (str, optional):
                Address of the kafka broker that a consumer
                should use. Defaults to be the same as `kafkaAddress`.
* kafka_address_producer (str, optional):
                Address of the kafka broker that a producer
                should use. Defaults to be the same as `kafkaAddress`.
* timeout (int, optional):
                Timeout value for GSQL queries, in ms. Defaults to 300000.

#### Load all vertices at once directly to local. Default.

In [5]:
%%time
vertex_loader = conn.gds.vertexLoader(
    num_batches=1, 
    attributes=["id", "x", "y"])
# Note: an unique ID of each vertex will be automatically generated and extracted. 
# If you want the primary ID of each vertex, specify it as an attribute.

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 208 ms, sys: 42.5 ms, total: 251 ms
Wall time: 37.9 s


In [6]:
%%time
# Get the only batch of data via the `data` property
data = vertex_loader.data
data.shape

CPU times: user 92.2 ms, sys: 28.7 ms, total: 121 ms
Wall time: 233 ms


(2708, 4)

In [7]:
data.head()

,vid,id,x,y
0,31457312,2374,0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 ...,5
1,6291542,1100,1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 ...,3
2,31457311,391,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,2
3,6291541,1800,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1
4,31457310,2355,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,4


#### Get batches of vertices through http

In [8]:
%%time
vertex_loader = conn.gds.vertexLoader( 
    num_batches=20, 
    attributes=["id", "x", "y"],
    shuffle=True,
    filter_by=None
)

CPU times: user 6.7 ms, sys: 139 µs, total: 6.84 ms
Wall time: 7.59 ms


In [9]:
%%time
for i, batch in enumerate(vertex_loader):
    print("----Batch {}: Shape {}----".format(i, batch.shape))
    print(batch.head(1))

----Batch 0: Shape (124, 4)----
        vid   id                                                  x  y
0  12582928  573  0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  5
----Batch 1: Shape (159, 4)----
        vid    id                                                  x  y
0  15728690  1412  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  2
----Batch 2: Shape (156, 4)----
        vid    id                                                  x  y
0  13631574  1810  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...  3
----Batch 3: Shape (138, 4)----
        vid  id                                                  x  y
0  20971543  20  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  5
----Batch 4: Shape (126, 4)----
        vid   id                                                  x  y
0  13631493  912  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  4
----Batch 5: Shape (128, 4)----
        vid   id                                                  x  y
0  31457292  255  0 0 0 0

#### Stream batches of vertices through Kafka

In [10]:
%%time
vertex_loader = conn.gds.vertexLoader(
    num_batches=20, 
    attributes=["id", "x", "y"],
    shuffle=True,
    filter_by=None,
    kafka_address="127.0.0.1:9092",
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 37.1 ms, sys: 11.9 ms, total: 48.9 ms
Wall time: 25.8 s


In [11]:
%%time
for i, batch in enumerate(vertex_loader):
    print("----Batch {}: Shape {}----".format(i, batch.shape))
    print(batch.head(1))

----Batch 0: Shape (157, 4)----
   vid    id                                                  x  y
0    9  1341  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  0
----Batch 1: Shape (121, 4)----
       vid    id                                                  x  y
0  7340080  2332  0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 ...  0
----Batch 2: Shape (131, 4)----
        vid   id                                                  x  y
0  11534338  562  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  2
----Batch 3: Shape (143, 4)----
        vid  id                                                  x  y
0  19922966  66  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 ...  4
----Batch 4: Shape (149, 4)----
       vid    id                                                  x  y
0  1048585  1443  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  4
----Batch 5: Shape (125, 4)----
       vid    id                                                  x  y
0  8388625  2477  0 0 0 0 0 0 0 0 0 0